In [1]:
# Check if GMST signal is projecting onto stdevs.

In [2]:
import numpy as np
import xarray as xr
from distributed import Client
import dask_jobqueue
#import dask
#import daskdataframe as dd
import matplotlib.pyplot as plt
import glob
import netCDF4 as nc
import zarr
import nc_time_axis
import xskillscore as xs
import cartopy as cart
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from scipy.special import erfinv, erf
#%run data_handler.ipynb
#np.set_printoptions(threshold=np.inf)

In [3]:
# #Chicago
# LAT        = 41.8781
# LON        = 360-87.6298
# # #Bengaluru:
LAT        = 12.9716
LON        = 77.5946
########## CONUS ############
#CONUS lat-lon
top     = 50.0 # north lat
left    = -124.7844079+360 # west long
right   = -66.9513812+360 # east long
bottom  =  24.7433195 # south lat
################################
cesm2_path = '/global/scratch/users/harsha/LENS/cesm2/tasmax/'
cvals      = '/global/scratch/users/harsha/LENS/cesm2/cvals/'
loc        = 'Bengaluru/'
locn       = 'Bengaluru'
cvals1     = '/global/scratch/users/harsha/LENS/cesm2/tmax_mem'
## Change/Check these parameters before running
# season     = 'JJA'
# season0    = 'jja'
# months     = [6,7,8]
# day        =  2111      #First day of the season
# Day        = 'Jul30'

In [4]:
def mean_adj(ds_stack,ds_mroll):
    ds = (ds_stack - ds_stack.mean(dim='mtime')) + ds_mroll
    return ds

def mean_std_adj(ds_stack,ds_mroll,ds_sroll):
    ds = (ds_stack -ds_stack.mean(dim='mtime'))*(ds_sroll/ds_stack.std(dim='mtime'))\
          + ds_mroll
    return ds

In [5]:
def gmst(arr, dims):
        #Compute GMST #g = global, a= annual, dims could be days, members etc
        weights         = np.cos(np.deg2rad(arr.lat))
        weights.name    = "weights"
        arr_weighted    = arr.weighted(weights)
        arr_gmean       = arr_weighted.mean({'lat','lon'},skipna=True)
        ####
        arr_gmst        = arr_gmean.mean(dims)       
        return arr_gmst

In [6]:
def to_daily(ds):
    year = ds.time.dt.year
    day = ds.time.dt.dayofyear

    # assign new coords
    ds = ds.assign_coords(year=("time", year.data), day=("time", day.data))

    # reshape the array to (..., "day", "year")
    return ds.set_index(time=("year", "day")).unstack("time")  

In [7]:
def stacker(x):
    #Assumes that x has dimensions year and member
    x = x.stack(ym=('year','member'),create_index=False)
    return x
def stacker2(x):
    #Assumes that x has dimensions year and member
    x = x.stack(mtime=('year','member'),create_index=False)
    return x

In [8]:
def stack_roll(ds,roll_len,stride_len):
    #Assumes that ds has dimensions year and member
    ds_stack   = stacker(ds)
    print(ds_stack)
    ds_roll    = ds_stack.rolling(ym=roll_len, center=True).construct('index', stride=stride_len)
    
    return ds_roll   

In [9]:
def global_annual_rmse(arr_err, dim):
      #Compute spatially weighted global mean of square error
      # and then compute annual rmse #g = global, a= annual
        arr_err_sq            = arr_err**2
        weights               = np.cos(np.deg2rad(arr_err_sq.lat))
        weights.name          = "weights"
        arr_serror_weighted   = arr_err_sq.weighted(weights)
        arr_serror_gmean      = arr_serror_weighted.mean({'lat','lon'},skipna=True)
        ####
        arr_gamse             = arr_serror_gmean.mean(dim)
        arr_garmse            = np.sqrt(arr_gamse)
        
        return arr_garmse

In [10]:
job_extra = ['--qos=cf_lowprio','--account=ac_cumulus'] 
#job_extra =['--qos=lr6_lowprio','--account=ac_cumulus','--constraint=lr6_m192']
#job_extra =['--qos=lr_lowprio','--account=ac_cumulus']
cluster = dask_jobqueue.SLURMCluster(queue="cf1", cores=10, walltime='6:00:00', 
                local_directory='/global/scratch/users/harsha/dask_space/', 
                log_directory='/global/scratch/users/harsha/dask_space/', 
                job_extra_directives=job_extra, interface='eth0', memory="192GB") 
client  = Client(cluster) 
cluster.scale_up(5)

In [11]:
cluster

In [12]:
pi_year0 = '1850'
pi_year1 = '1879'
ic_year0 = '2071'
ic_year1 = '2100'
#doy      = 211 # day_of_year
#date     = 'Jul30'

In [13]:
%%time
ds_ssp  = xr.open_mfdataset(cesm2_path+'ssp370_cesm2tmax.zarr',engine='zarr')
ds_hist = xr.open_mfdataset(cesm2_path+'bhist_cesm2tmax.zarr',engine='zarr')

CPU times: user 1.31 s, sys: 356 ms, total: 1.67 s
Wall time: 6.53 s


In [14]:
%%time
#t-temp, nw - no-warming, w-warming, hw = historical warming, g=global
tgw_eoc = ds_ssp.TREFHTMX.sel(time=slice(ic_year0,ic_year1))
         #.sel(lat=slice(bottom,top))
         #.sel(lon=slice(left,right))
tgnw   = ds_hist.TREFHTMX.sel(time=slice(pi_year0,pi_year1))
         #.sel(lon=slice(left,right)).sel(lat=slice(bottom,top))
tghw   = ds_hist.TREFHTMX
         #.sel(lat=slice(bottom,top))
         #.sel(lon=slice(left,right))
tgnw

CPU times: user 180 ms, sys: 11.5 ms, total: 191 ms
Wall time: 185 ms


,Array,Chunk
Bytes,225.56 GiB,769.92 MiB
Shape,"(10950, 192, 288, 100)","(3650, 192, 288, 1)"
Count,2001 Tasks,300 Chunks
Type,float32,numpy.ndarray


In [15]:
tgw_pid  = to_daily(tgnw)
tgw_eocd = to_daily(tgw_eoc)
tgw_eocd = tgw_eocd.chunk({'lat':20,'lon':20})
tgw_pid  = tgw_pid.chunk({'lat':20,'lon':20})
tgw_pid

/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/dataset.py:4880: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  result = result._unstack_full_reindex(
/global/home/users/harsha/miniconda3/envs/pyenv/lib/python3.9/site-packages/xarray/core/dataset.py:4880: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

,Array,Chunk
Bytes,225.56 GiB,5.57 MiB
Shape,"(192, 288, 100, 30, 365)","(20, 20, 1, 10, 365)"
Count,92601 Tasks,45000 Chunks
Type,float32,numpy.ndarray


In [16]:
eoc_gmst  = gmst(tgw_eocd,{'day','member'})
pi_gmst   = gmst(tgw_pid,{'day','member'})
eoc_gmst

<xarray.DataArray 'TREFHTMX' (year: 30)>
dask.array<mean_agg-aggregate, shape=(30,), dtype=float64, chunksize=(10,), chunktype=numpy.ndarray>
Coordinates:
  * year     (year) int64 2071 2072 2073 2074 2075 ... 2096 2097 2098 2099 2100

In [17]:
%%time
# eoc_gmst.plot()

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 13.4 µs


In [18]:
%%time
# pi_gmst.plot()

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 14.1 µs


In [19]:
#eoc_gmsta.values
#
# eoc_gmst_change = eoc_gmst.sel(year= 2100) - eoc_gmst.sel(year=2071)
# eoc_gmst_change.values

In [20]:
########## Detrending daily tasmax series ###############

In [21]:
%%time
pcoeffs_eoc = tgw_eocd.mean('member').polyfit(dim='year',deg=1)
pcoeffs_pi  = tgw_pid.mean('member').polyfit(dim='year',deg=1)
pcoeffs_eoc = pcoeffs_eoc.chunk({'lat':10,'lon':12})
pcoeffs_pi  = pcoeffs_pi.chunk({'lat':10,'lon':12})
pcoeffs_eoc

CPU times: user 4.09 s, sys: 3.11 s, total: 7.2 s
Wall time: 6.67 s


,Array,Chunk
Bytes,307.97 MiB,684.38 kiB
Shape,"(2, 192, 288, 365)","(2, 10, 12, 365)"
Count,207838 Tasks,480 Chunks
Type,float64,numpy.ndarray


In [22]:
# Use polyfit to get the slope (m) and intercept (c)
m_eoc         = pcoeffs_eoc.polyfit_coefficients.sel(degree=1)
c_eoc         = pcoeffs_eoc.polyfit_coefficients.sel(degree=0)

# Calculate the corrected intercept (c') to make sure the y-intercept at 2085 is zero
c_prime_eoc = -m_eoc * 2085 - c_eoc

# Compute the trend using m and c_prime
trend_eoc = m_eoc * tgw_eocd['year'] + c_prime_eoc

# Detrend the data
detrend_tgw_eoc = tgw_eocd - trend_eoc
detrend_tgw_eoc

,Array,Chunk
Bytes,451.13 GiB,3.34 MiB
Shape,"(192, 288, 100, 30, 365)","(10, 12, 1, 10, 365)"
Count,1035519 Tasks,272000 Chunks
Type,float64,numpy.ndarray


In [23]:
# Use polyfit to get the slope (m) and intercept (c)
m_pi        = pcoeffs_pi.polyfit_coefficients.sel(degree=1)
c_pi        = pcoeffs_pi.polyfit_coefficients.sel(degree=0)

# Calculate the corrected intercept (c') to make sure the y-intercept at 2085 is zero
c_prime_pi = -m_pi * 1865 - c_pi

# Compute the trend using m and c_prime
trend_pi = m_pi * tgw_pid['year'] + c_prime_pi

# Detrend the data
detrend_tgw_pi = tgw_pid - trend_pi
detrend_tgw_pi

,Array,Chunk
Bytes,451.13 GiB,3.34 MiB
Shape,"(192, 288, 100, 30, 365)","(10, 12, 1, 10, 365)"
Count,779699 Tasks,204000 Chunks
Type,float64,numpy.ndarray


In [24]:
%%time
#Stack member and year into one dimension- 'ym'. Then, roll over this dimension with stride = N_mem.
#This works because of the way xarray stacks two dimensions. window_len =30. I have called the 'window' dimension 'index'.
quants             = np.arange(0,1.01,0.01)
########################
N_mem              = 100
window_len         = 30
###### season #######
#r = roll, ndrop = drop NANs
tgmw_pidetroll    = stack_roll(detrend_tgw_pi,N_mem*window_len,N_mem)
tgmw_eocdetroll   = stack_roll(detrend_tgw_eoc,N_mem*window_len,N_mem)
#########
tgmw_pidetroll0   = tgmw_pidetroll.sel(ym=[15])  
tgmw_eocdetroll0  = tgmw_eocdetroll.sel(ym=[15])  
tgmw_pidetroll0

<xarray.DataArray (lat: 192, lon: 288, day: 365, ym: 3000)>
dask.array<reshape, shape=(192, 288, 365, 3000), dtype=float64, chunksize=(10, 12, 365, 100), chunktype=numpy.ndarray>
Coordinates:
    year     (ym) int64 1850 1850 1850 1850 1850 ... 1879 1879 1879 1879 1879
  * day      (day) int64 1 2 3 4 5 6 7 8 9 ... 358 359 360 361 362 363 364 365
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member   (ym) int64 0 1 2 3 4 5 6 7 8 9 10 ... 90 91 92 93 94 95 96 97 98 99
    degree   int64 1
Dimensions without coordinates: ym
<xarray.DataArray (lat: 192, lon: 288, day: 365, ym: 3000)>
dask.array<reshape, shape=(192, 288, 365, 3000), dtype=float64, chunksize=(10, 12, 365, 100), chunktype=numpy.ndarray>
Coordinates:
    year     (ym) int64 2071 2071 2071 2071 2071 ... 2100 2100 2100 2100 2100
  * day      (day) int64 1 2 3 4 5 6 7 8 9 ... 358 359 360 361 362 363 364 365
  * lat

,Array,Chunk
Bytes,451.13 GiB,0.98 GiB
Shape,"(192, 288, 365, 1, 3000)","(10, 12, 365, 1, 3000)"
Count,1124140 Tasks,680 Chunks
Type,float64,numpy.ndarray


In [25]:
%%time
#Compute quantiles 
qgmw_pidet        = tgmw_pidetroll0.quantile(quants,dim='index',skipna=False)
qgmw_eocdet       = tgmw_eocdetroll0.quantile(quants,dim='index',skipna=False)
qgmw_pidet        = qgmw_pidet.rename('pi_qdet')
qgmw_eocdet        = qgmw_pidet.rename('eoc_qdet')
qgmw_pidet        = qgmw_pidet.chunk({'lon':50})
qgmw_eocdet       = qgmw_eocdet.chunk({'lon':50})
qgmw_pidet

CPU times: user 38.1 ms, sys: 1.22 ms, total: 39.4 ms
Wall time: 37.5 ms


<xarray.DataArray 'pi_qdet' (quantile: 101, lat: 192, lon: 288, day: 365, ym: 1)>
dask.array<rechunk-merge, shape=(101, 192, 288, 365, 1), dtype=float64, chunksize=(101, 10, 50, 365, 1), chunktype=numpy.ndarray>
Coordinates:
    year      (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * day       (day) int64 1 2 3 4 5 6 7 8 9 ... 358 359 360 361 362 363 364 365
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    member    (ym) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * quantile  (quantile) float64 0.0 0.01 0.02 0.03 0.04 ... 0.97 0.98 0.99 1.0
Dimensions without coordinates: ym

In [ ]:
#### season ######
qgmw_pidetpath   = cvals+'detrended/qgmw_pidet'+pi_year0+'_'+pi_year1+'.zarr'
qgmw_eocdetpath  = cvals+'detrended/qgmw_eocdet'+ic_year0+'_'+ic_year1+'.zarr'
print(qgmw_pidetpath)
qgmw_pidet.to_dataset().to_zarr(qgmw_pidetpath,mode='w')

/global/scratch/users/harsha/LENS/cesm2/cvals/detrended/qgmw_pidet1850_1879.zarr


In [ ]:
%%time
qgmw_eocdet.to_dataset().to_zarr(qgmw_eocdetpath,mode='w')